In [1]:
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
all_amz_go = pd.read_csv('F:\\mailae_g\\ALL_MAG_2021+2022.csv', sep=';' )
all_cus_go = pd.read_csv('F:\\mailae_g\\ALL_PL_2022.csv', sep=';')

In [4]:
all_amz_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,16477,Frage von Amazon-Kunde Anja Klose,Skraut Home- Glanzlack Wohnwand. Von dem großen hängeschrank der Boden ist beschädigt. Beim Dübel loch ist was abgebrochen.,Email,freshdeskuser_18653026@channelreply.email,13 09 2021 15:58,26,18,"lukfurniture_de,cr,lukfurniture,rynek_de"
1,83046,Frage von Amazon-Kunde Furkan Aydin(Bestellung: 306-1709843-3097134),Hallo Die schiene von der schublade ist kaputt da fallen die kleinen Kugeln raus. Ich möchte gerne 2 schienen.,Email,freshdeskuser_20098643@channelreply.email,7 12 2021 22:56,12,6,"lukfurniture_de,cr,lukfurniture,rynek_de"
2,92323,Return policy enquiry from Amazon customer Mrs Susan Tolley(Order: 204-9663987-5997940),Hi I purchased the welder back in April 2021 but due to ill health I have only been able to try the welder today. Unfortunately It does not work. I think there maybe a problem with the MIG output to the switch on the gun. I have used a test meter on the contacts and have no output. Please can you advise. Thankyou.,Email,freshdeskuser_20274812@channelreply.email,16 12 2021 13:51,65,40,"cr,rynek_uk,allweld,allweld_uk"
3,165940,Frage von Amazon-Kunde Fjolla(Bestellung: 306-7063158-3705902),Sehr Geehrte Damen und Herren Es geht um meine Bestellung die ich seit Wochen gemacht habe.Auf meine Bestellung steht dass mein Paket möglicherweise verloren gegangen ist.Ich habe dafür Geld bezahlt.Wie können sie mir weiter helfen? Mit Freundliche Grüßen,Email,freshdeskuser_21788405@channelreply.email,19 03 2022 17:47,14,9,"lukfurniture_de,cr,lukfurniture,rynek_de"
4,192109,"? Sie haben einen neuen Preisvorschlag: EUR 60,00 für NeoNail Nagelstudio ... (373940992894)","Preisvorschlag: EUR 60,00 Preisvorschlag: EUR 60,00 Jetzt antworten: https://www.ebay.de/ulk/showOffers/373940992894?boolp=1&amp;mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44861&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Artikel ansehen: https://www.ebay.de/itm/373940992894?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44862&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=45010273334&amp;osub=-1%7E1&amp;crd=20220424091836&amp;segname=12060&amp;sojTags=ch%3Dch%2Cbu%3Dbu%2Cosub%3Dosub%2Ccrd%3Dcrd%2Csegname%3Dsegname%2Cchnl%3Dmkcid Unter Angebotspreis EUR 4,89 Dieses Angebot sichern 9 Beobachter meliba601: https://www.ebay.de/ulk/usr/meliba601?mkevt=1&amp;mkpid=0&amp;emsid=e12060.m3112.l44863&amp;mkcid=26&amp;ch=osgood&amp;euid=0fe5676504234b2c849a719471b7060f&amp;bu=4...",Email,freshdeskuser_22171370@channelreply.email,24 04 2022 18:56,1,12,"ServiceMessage,cr,gashio,gashio_de_eb,ebay,rynek_de_eb"


In [5]:
all_cus_go.head(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
0,111345,[DE] [PYTANIE] B07R42W8J9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,shop@folnet.de,10 01 2022 10:47,4,4,"Partner_wysłane,rynek_de"
1,128291,[DE] 302-0014409-7188317 NIEKOMPLETNE,"Klient zgłasza, że w zamówieniu nie było lampki. Pozdrawiam, Jakub",Outbound Email,info.palulli@gmail.com,30 01 2022 09:48,3,2,"Partner_wysłane,rynek_de"
2,128325,[REKLAMACJA] 205-9594094-3558723 UK,"Dzień dobry, Klient napisał: Ten produkt dotarł, ale nie działa, nie świeci się. Bardzo rozczarowany, ponieważ kupiłem go dla mojego dziecka. Brak zauważalnych wad, wszystko wydawało się w porządku po otwarciu. Mały przycisk przesuwa się w dół i wydaje się OK, ale nic się nie dzieje, gdy go nacisnąć. To nie świeci się, co było cały punkt z nim, i bardzo rozczarowujące dla mojej córki. Pozdrawiam, Hania",Outbound Email,polskaksiegarniainternetowa@gmail.com,30 01 2022 10:38,2,2,"Partner_wysłane,rynek_uk,Przypomnienie"
3,131092,[DE] [ZAPYTANIE] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,kotarbauzapytania@gmail.com,2 02 2022 16:10,4,4,"Partner_wysłane,rynek_de,Przypomnienie"
4,133983,[IT] 408-8392161-3066735 ZWROT,"https://www.dhl.com/it-en/home/tracking/tracking-parcel.html?submit=1&amp;tracking-id=CM332431606DE Paczka wraca do was, prosimy o refund dla klienta jak dojdzie Kasia",Outbound Email,kotarbauzapytania@gmail.com,7 02 2022 08:58,9,6,"Partner_wysłane,rynek_it,Przypomnienie1,Przypomnienie2"


In [6]:
all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [7]:
# Funkcja do dzielenia komórek na listę, pierwszy argument tekst, drugi argument po czym tekst ma być dzielony 

def split_tags(tags: str, split_char: str) -> list:
    try:
        tags = tags.split(f'{split_char}')
        return tags 
    except Exception:
        return (list[tags])
    

In [8]:
# funkcja do wyciągania z listy, jakiego rynku dotyczą wiadomości

def get_market(lst: list) -> str:
    for i in lst:
        if "rynek" in i:
            return i

In [9]:
# funkcja do wyciągania numerów zamówień z tekstu i tytułu maila. Przyjmuje dwa różne ciągi tekstowe, sprawdza czy jest w tytule jest regex, jeśli nie to sprawdza czy jest w opisie, jeśli nie to zwraca pustą listę

se = re.compile(r"\d{3}-\d{7}-\d{7}")

def order_number(subject: str, description: str) -> list:
    subject = str(subject)
    description = str(description)
    if se.findall(subject) == []:
        return se.findall(description)
    return se.findall(subject)


In [10]:
# funkcja do tworzenia listy unikalnych wartości z kolumn. Dajemy wartość z kolumny i listę. Sprawdza czy wartość już znajduje się na liście, jeśli nie to ją dodaje

def all_order(order: list, new_list: list) -> list:
    try:
        if order[0] in new_list:
            return new_list
        else:
            new_list.append(order)
            return new_list
    except Exception:
        i =1

In [11]:
# druga funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

on = re.compile(r"\d{3}-\d{7}-\d{7}")
cc = re.compile(r"w{2}")
cc2 = re.compile(r'\[\w{2}\]')

def clear_title(title_list: list) -> list:
    for el in title_list:
        print(el)
    list_to_remove = []
    for i in range(len(title_list)):
        if re.match(on, title_list[i]):
            title_list.remove(i)
        elif re.match(cc, title_list[i]):
            title_list.remove(i)
        elif re.match(cc2, title_list[i]):
            title_list.remove(i)
    return title_list

In [12]:
# trzecia funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def num_there(s):
    return any(i.isdigit() for i in s)

def find_words_with_more_than_four_characters(words_with_tags):
  words = []
  for i in words_with_tags:
    if num_there(i):
      continue
    w = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', i)
    w = w.strip(" ")
    if len(w) >= 4:
      words.append(w)

  return words

In [13]:
# czwarta funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def find_words_with_more_than_four_characters2(subject: str) -> list:
  words = []
  new_subject = ''
  for l in subject:
    l = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', l)
    new_subject += l
  words_temp = new_subject.split(' ')
  for i in words_temp:
    if len(i) >= 4:
      words.append(i)

  return words

In [14]:
# Funkcja do zliczania elementów w liscie, zwraca listę unikatowych wartości i listę unikatowych wartości z informacją ile razy występowały

from collections import Counter

def count_occurrences(input_list):
    unique_values = set(input_list)
    occurrence_count = Counter(input_list)

    return list(unique_values), occurrence_count

In [15]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [16]:
#

from itertools import chain

def get_all_values (dict1: dict, dict0: dict):
    final_dict = {}
    key_lvl1 = []
    key_lvl1.append(list(dict1.keys()))
    key_lvl1 = list(chain(*key_lvl1))
    not_working_keys = []
    for i in key_lvl1:
        temp_vals = []
        temp_keys = dict1[i]
        temp_keys = [x for x in temp_keys if x is not np.nan]
        for j in temp_keys:
            try:
                temp_vals.append(list(dict0[j]))
            except Exception:
                    not_working_keys.append(j)
        temp_vals = set(item for sublist in temp_vals for item in sublist)
        final_dict[i] = temp_vals
    return not_working_keys, final_dict

In [17]:
def get_all_values2(dict1: dict, dict0: dict):
    final_dict = {}
    not_working_keys = []

    for key, values in dict1.items():
        valid_values = [v for v in values if v is not np.nan]
        temp_vals = []

        for v in valid_values:
            try:
                temp_vals.extend(dict0[v])
            except Exception:
                not_working_keys.append(v)

        final_dict[key] = set(temp_vals)

    return not_working_keys, final_dict

In [18]:
def title_classification (dict_of_words: dict, title: list) -> str:
    if len(title) > 0:
        for key, values in dict_of_words.items():
            for i in title:
                if i in values:
                    return key

In [19]:
# Zmiana tytułów na lowercase

all_amz_go['Subject'] = [str(row).lower() for row in all_amz_go['Subject']]
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]

In [20]:
# przekształcenie stringów w komórkach kolumny tags na listę, dzielone po przecinku 

all_amz_go['Tags'] = [split_tags(row, ',')for row in all_amz_go['Tags']]
all_cus_go['Tags'] = [split_tags(row, ',')for row in all_cus_go['Tags']]

In [21]:
# utworzenie kolumny, w której jest napisane jakiego rynku dotyczy wiadomość, na podstawie kolumny tags

all_amz_go['rynek'] = [get_market(row) for row in all_amz_go['Tags']]
all_cus_go['rynek'] = [get_market(row) for row in all_cus_go['Tags']]

In [22]:
# usunięcie wierszy, które w kolumnie 'rynek' nie maję 'rynek_de'

# all_amz_go.drop(all_amz_go[all_amz_go['rynek'] != 'rynek_de'].index,  inplace=True)
# all_cus_go.drop(all_cus_go[all_cus_go['rynek'] != 'rynek_de'].index,  inplace=True)

In [23]:
# wyciągnięcie numeru zamówień z kolumn subject i description

all_amz_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_amz_go['Subject'], all_amz_go['Description'])]
all_cus_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]

In [24]:
# resetowanie indexów

all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [25]:
# zmiana obiektów z kolummny 'Created Time' na daty

from datetime import datetime

format_date = "%d %m %Y %H:%M"

all_amz_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_amz_go['Created Time']] 
all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 

In [26]:
# wykorzystanie funkcji do splitowania aby utworzyć kolumnę 

all_amz_go['title_list'] = [split_tags(row, ' ') for row in all_amz_go['Subject']]
all_cus_go['title_list'] = [split_tags(row, ' ') for row in all_cus_go['Subject']]

In [27]:
# sprawdzenie czy funcja split_tags zadzaiała poprawnie

all_cus_go['title_list'][1]

['[de]', '302-0014409-7188317', 'niekompletne']

W tym momencie postanowieł sprawdzić, czy zamówienia, z jednej i drugiej tabeli się pokrywają w jakimś najmniejszym stopniu

In [28]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy jednego data set

all_amz_go_list = []
[all_order(row, all_amz_go_list) for row in all_amz_go['Order_number']]
all_amz_go_list = [item for sublist in all_amz_go_list for item in sublist]

In [29]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy drugiego data set

all_cus_go_list = []
[all_order(row, all_cus_go_list) for row in all_cus_go['Order_number']]
all_cus_go_list = [item for sublist in all_cus_go_list for item in sublist]

In [30]:
# zamieniam stworzyonych list na set

mcg_order_set = set(all_amz_go_list)
mag_rynek_de_order_set = set(all_cus_go_list)

In [31]:
# dodanie kontrolnie jednej takiej samej wartości do obu setów

mcg_order_set.add('123')
mag_rynek_de_order_set.add('123')

In [32]:
# porównanie dwóch

y = mcg_order_set.intersection(mag_rynek_de_order_set)

In [33]:
# Wyświetlenie ile mamy takich samych numerów zamówień w jednym i drugim secie

len(y)

11037

Powrót do preprocesingu danych

In [34]:
# weryfickaja działania funkcji do czyszczenia tytułów maili

print(find_words_with_more_than_four_characters2(all_cus_go['Subject'][5552]))

['zapytanie', 'amandine', 'perez']


In [35]:
# utworzenie nowej kolumny label i dodanie do niej słów, które nadają się na labelki

all_cus_go['label'] = [find_words_with_more_than_four_characters2(row) for row in all_cus_go['Subject']]

In [36]:
# sprawdzenie jak wygląda labels dla 10 przykładowych wierszy

all_cus_go['label'].head(10)

0                    [pytanie]
1               [niekompletne]
2                 [reklamacja]
3                  [zapytanie]
4                      [zwrot]
5          [zwrot, reklamacja]
6                 [reklamacja]
7                 [reklamacja]
8    [zapytanie, asin, bfwdtm]
9                    [pytanie]
Name: label, dtype: object

In [37]:
# Stworzenie listy ze wszystkimi wartościami z kolumny labels

label_list = []
[all_order(row, label_list) for row in all_cus_go['label']]
label_list = [item for sublist in label_list for item in sublist]

In [38]:
# Stworzenie dwóch list na podstawie listy wartości z kolumny['label'], jedna lista to unikatowe wartości, druga lista unikatowe wartości z informacją ile dana wartość występowała

uniq_vales, uniq_values_count = count_occurrences(label_list)

In [39]:
if 'anulownanie' in uniq_vales:
    print('yes')

yes


In [40]:
# Stworzenie obiektu spacy z polskim słownikiem

nlp = spacy.load("pl_core_news_lg")

In [41]:
# Zmiana unikatowej listy na string

uniq_vales_sentence = ' '.join(uniq_vales)

In [42]:
if 'anulownanie' in uniq_vales_sentence:
    print('yes')

yes


In [43]:
# tokenizacja stringu unikatowych wartości z kolumny tags

doc = nlp(uniq_vales_sentence)

In [44]:
# lemantyzacja wcześniej stworzonych tokenów. Stworzenie listy i listy tupli. Lista tupli zawierające słowo i jej podstawę słowotwórczą, sama lista zawiera wszystkie podstwy słowotwórcze 

all_lem = []
all_words = []
for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

In [45]:
# import pickle
# with open('all_lem.pkl', 'wb') as file:
#     pickle.dump(all_lem, file)

In [46]:
all_lem_df = pd.DataFrame(all_lem)
all_words_df = pd.DataFrame(all_words)
all_lem_df.to_csv('all_lem_df7.csv', sep=',', encoding='cp1250',index=False)
all_words_df.to_csv('all_words_df6.csv', sep=',', encoding='cp1250',index=False)

In [47]:
# Stworzenie słownika gdzie klucz to podstawa słowotwórcza a values jest listą słów dla tej podstawy występująca w kolumnie tags

lem_dict_0 = {}
for i in set(all_lem):
    lem_dict_0[i] = find_all_values_to_dictonary(i,all_words)

In [48]:
# stworzenie słownika ze specjalnego obieku counter

uniq_values_count_dict = dict(uniq_values_count)

In [49]:
# Stworzenie słownika, gdzie klucz to podstawa słwowtwórcza, a values to suma wartości występowania słów zawiązanych z tą lemą

number_of_occurence = {}
for i in lem_dict_0:
    number_of_occurence_value = []
    for j in lem_dict_0[i]:
        number_of_occurence_value.append(uniq_values_count_dict[j])
    number_of_occurence[i] = sum(number_of_occurence_value)

In [50]:
# Posortowanie słownika zawierającego podstawę słowotwórczą oraz ilość wystąpnień słów z nim powiązanych

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [51]:
labels_df = pd.read_csv('labels2.csv', sep=';', header=None)
labels_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,anulacja,anulować,anulacja,anulowac,anulacj,anulownie,anulownaie,nieanulowanie,anulaj,anuloewanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reklamacja,reklamacja,reklamation,reklamcja,reklaamcja,reklamajci,reklmacja,reklacja,reklama,reklamationa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wysyłka,wysyłce,wysyłka,wysyłke,wysyp,wysylce,wysyłkąć,wysyka,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refund,refund,refundacja,refunda,refunf,refung,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,przesyłka,przesyłka,przesylka,przesyłewka,przesyłkami,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
labels_df = labels_df.T
labels_df.columns = labels_df.iloc[0]
label_name = labels_df.iloc[0]
labels_df = labels_df[1:]

In [53]:
lem_dict_1 = labels_df.to_dict(orient='list')

In [54]:
not_working_keys, final_dict = get_all_values(lem_dict_1, lem_dict_0)

In [55]:
len(not_working_keys)

59

In [56]:
all_cus_go['final_label'] = [title_classification(final_dict, row) for row in all_cus_go['label']]

In [57]:
all_cus_go['final_label'].value_counts()

final_label
dostawa         6547
zwrot           5884
anulacja        5408
reklamacja      4640
zapytanie       2631
faktura         1858
wysyłka         1303
paczka          1256
adres            957
brak             803
refund           776
uszkodzony       699
przesyłka        411
niekompletne     395
opóźnienie       299
ponowna          159
inne             127
Name: count, dtype: int64

In [58]:
df_to_model = pd.DataFrame()
for i in all_cus_go['final_label'].unique():
    subset = all_cus_go[all_cus_go['final_label'] == i].head(395).reset_index(drop=True)
    df_to_model = pd.concat([df_to_model, subset], axis=0)

In [59]:
df_to_model['final_label'].value_counts()

final_label
zapytanie       395
niekompletne    395
anulacja        395
przesyłka       395
paczka          395
refund          395
wysyłka         395
dostawa         395
adres           395
faktura         395
brak            395
uszkodzony      395
zwrot           395
reklamacja      395
opóźnienie      299
ponowna         159
inne            127
Name: count, dtype: int64

In [60]:
df_to_model

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags,rynek,Order_number,title_list,label,final_label
0,111345,[de] [pytanie] b07r42w8j9,Klient ma pytanie do produktu B07R42W8J9: &quot;Jakie połączenie ma dachówka wentylacyjna?&quot; Link do zapytania: www.amazon.de/ask/question/Tx21NZNJ8PMI5JH?_encoding=UTF8&amp;authToken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beata,Outbound Email,shop@folnet.de,2022-01-10 10:47:00,4,4,"[Partner_wysłane, rynek_de]",rynek_de,[],"[[de], [pytanie], b07r42w8j9]",[pytanie],zapytanie
1,131092,[de] [zapytanie] 305-6969626-5165121,"Klient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. Pozdrawiam Beata",Outbound Email,kotarbauzapytania@gmail.com,2022-02-02 16:10:00,4,4,"[Partner_wysłane, rynek_de, Przypomnienie]",rynek_de,[305-6969626-5165121],"[[de], [zapytanie], 305-6969626-5165121]",[zapytanie],zapytanie
2,172784,[zapytanie o asin] b09bfwdtm7 de,"Klient pyta o B09BFWDTM7 Jak wysoko znajduje się rama zagłowia? A kolor jest szarawy, czy może coś pomiędzy szarym a beżowym? https://www.amazon.de/ask/question/TxKRK621U2ZKVZ?_encoding=UTF8&amp;authToken=8650b5ee-b40b-4643-9d22-04d369cfc85c&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Małgorzata",Outbound Email,gonito@lukmebel.pl,2022-03-29 13:22:00,2,1,"[Partner_wysłane, rynek_de, Przypomnienie1]",rynek_de,[],"[[zapytanie, o, asin], b09bfwdtm7, de]","[zapytanie, asin, bfwdtm]",zapytanie
3,178772,[de] [pytanie] b0816rn2dl,Klient ma pytanie do produktu B0816RN2DL: &quot;Czy rama metalowa jest sztywna czy elastyczna?&quot; Link: www.amazon.de/ask/question/Tx3N1M04GDRF4JX?_encoding=UTF8&amp;authToken=80c83023-4c8a-42f0-8fab-d3c258a2ce5b&amp;ref_=s_respond&amp;answerSolicitationSourceType=EMAIL&amp;answerSolicitationLanguage=de_DE Pozdrawiam Beataa,Outbound Email,shop@folnet.de,2022-04-05 16:11:00,2,2,"[Partner_wysłane, rynek_de]",rynek_de,[],"[[de], [pytanie], b0816rn2dl]",[pytanie],zapytanie
4,210203,[usa] pytanie o produkt #391147462,"Klient pyta czy ten produkt będzie kiedyś dostępny: &quot;THE BATTLE OF GRUNWALD, JAN MATEJKO PUZZLE 4000 PIECES Box size: 46.8×27.2×5 cm Puzzle size: 138×68 cm EAN: 5904438400331&quot; Przekazuje też zdęcie. Pozdrawiam, Anita",Outbound Email,gonito@fkwt.pl,2022-05-17 13:50:00,2,2,"[Partner_wysłane, rynek_us]",rynek_us,[],"[[usa], pytanie, o, produkt, #391147462]","[pytanie, produkt]",zapytanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,3111216,[de] [ponowna] 302-2165623-5391540,"Witam, klient prosi o ponowną wysyłkę Brückenstraße 76, 15562 Rüdersdorf bei Berlin, Deutschland Pozdrawiam Joanna",Outbound Email,NaN,2023-04-09 09:05:00,2,2,"[Partner_wysłane, rynek_de]",rynek_de,[302-2165623-5391540],"[[de], [ponowna], 302-2165623-5391540]",[ponowna],ponowna
155,3115057,de [ponowna] 303-2351818-4778757,"Witam, paczka prawdopodobnie trafi do państwa z powrotem. Klient pyta, czy możlowa jest ponowna wysyłka? Pozdrawiam Joanna",Outbound Email,NaN,2023-04-18 18:12:00,2,2,"[rynek_de, B11]",rynek_de,[303-2351818-4778757],"[de, [ponowna], 303-2351818-4778757]",[ponowna],ponowna
156,3098507,[de] [ponowna] 028-1514291-6976328,"Witam, klient nie odebrał przesyłki w ciągu 7 dni, prosi o ponowną wysyłkę: Jacek Kaluza Himmelohstrasse 18e 58454 Witten Pozdrawiam Joanna",Outbound Email,NaN,2023-03-12 08:32:00,1,1,"[Partner_wysłane, rynek_de]",rynek_de,[028-1514291-6976328],"[[de], [ponowna], 028-1514291-6976328]",[ponowna],ponowna
157,3084889,[de] [ponowna] 305-4474672-2788349,"Witam, klient zupełnie nie rozumie, czemu nie można był dostarczyć jego przesyłki. Potwierdza adres: Aniela Bejna Berlinerstrasse 290 63067 Offenbach Am Main I prosi o ponowną wysyłkę. Zaznacza, że prosi o dostarczenie pomiędzy godz. 16-18 

In [61]:
messeges = ["".join(row.strip()) for row in df_to_model['Description']]
messeges = ''.join(messeges).lower().strip()

In [62]:
len(messeges)

2363096

In [63]:
messeges

'klient ma pytanie do produktu b07r42w8j9: &quot;jakie połączenie ma dachówka wentylacyjna?&quot; link do zapytania: www.amazon.de/ask/question/tx21nznj8pmi5jh?_encoding=utf8&amp;authtoken=967c969c-3f2e-47dd-b5e6-faeb39d052d1&amp;ref_=s_respond&amp;answersolicitationsourcetype=email&amp;answersolicitationlanguage=de_de pozdrawiam beataklient chciałby wiedzieć, czy materiał, z którego wykonane są klamki może być stosowany również na zewnątrz, np. na bocznych drzwiach wejściowych. pozdrawiam beataklient pyta o b09bfwdtm7\xa0 jak wysoko znajduje się rama zagłowia? a kolor jest szarawy, czy może coś pomiędzy szarym a beżowym? https://www.amazon.de/ask/question/txkrk621u2zkvz?_encoding=utf8&amp;authtoken=8650b5ee-b40b-4643-9d22-04d369cfc85c&amp;ref_=s_respond&amp;answersolicitationsourcetype=email&amp;answersolicitationlanguage=de_de pozdrawiam małgorzataklient ma pytanie do produktu b0816rn2dl: &quot;czy rama metalowa jest sztywna czy elastyczna?&quot; link: www.amazon.de/ask/question/tx3n

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [65]:
X = df_to_model['Description']
y = df_to_model['final_label']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [67]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [68]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [69]:
X_train_tf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [70]:
tfidf.get_feature_names_out()

array(['00', '000', '0000', ..., 'żądania', 'żądany', 'żę'], dtype=object)

In [71]:
pd.DataFrame(X_train_tf.todense(),
             columns=tfidf.get_feature_names_out())

,00,000,0000,000000992432,0000028368,0000047237,0000083531,0000142805,0000357766,0000390511,...,żółte,żółtej,żółto,żółty,żółtym,żąda,żądanej,żądania,żądany,żę
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_tf, y_train)

LogisticRegression()

In [73]:
y_pred = lr.predict(X_test_tf)

In [74]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.73      0.86      0.79        80
    anulacja       0.94      0.84      0.89        76
        brak       0.72      0.65      0.69        75
     dostawa       0.71      0.76      0.74        75
     faktura       1.00      0.93      0.96        82
        inne       0.91      0.56      0.69        18
niekompletne       0.74      0.90      0.81        87
  opóźnienie       0.72      0.59      0.65        61
      paczka       0.86      0.63      0.73        87
     ponowna       0.87      0.79      0.83        33
   przesyłka       0.77      0.78      0.77        68
      refund       0.81      0.85      0.83        68
  reklamacja       0.62      0.65      0.63        77
  uszkodzony       0.73      0.73      0.73        83
     wysyłka       0.58      0.79      0.67        72
   zapytanie       0.81      0.87      0.84        92
       zwrot       0.75      0.62      0.68        89

    accuracy              

In [75]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)

y_pred = rfc.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.72      0.81      0.76        80
    anulacja       0.94      0.86      0.90        76
        brak       0.76      0.64      0.70        75
     dostawa       0.66      0.76      0.71        75
     faktura       0.99      0.96      0.98        82
        inne       0.83      0.56      0.67        18
niekompletne       0.78      0.86      0.82        87
  opóźnienie       0.77      0.56      0.65        61
      paczka       0.86      0.78      0.82        87
     ponowna       0.89      0.76      0.82        33
   przesyłka       0.70      0.75      0.72        68
      refund       0.83      0.87      0.85        68
  reklamacja       0.58      0.55      0.56        77
  uszkodzony       0.70      0.75      0.72        83
     wysyłka       0.71      0.83      0.77        72
   zapytanie       0.79      0.85      0.82        92
       zwrot       0.72      0.71      0.72        89

    accuracy              

In [76]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

y_pred = mnb.predict(X_test_tf)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       adres       0.82      0.62      0.71        80
    anulacja       0.88      0.84      0.86        76
        brak       0.54      0.29      0.38        75
     dostawa       0.54      0.75      0.63        75
     faktura       0.94      0.93      0.93        82
        inne       0.91      0.56      0.69        18
niekompletne       0.72      0.94      0.82        87
  opóźnienie       0.46      0.43      0.44        61
      paczka       0.80      0.51      0.62        87
     ponowna       1.00      0.24      0.39        33
   przesyłka       0.51      0.59      0.54        68
      refund       0.76      0.75      0.76        68
  reklamacja       0.76      0.57      0.65        77
  uszkodzony       0.63      0.82      0.71        83
     wysyłka       0.36      0.78      0.49        72
   zapytanie       0.94      0.83      0.88        92
       zwrot       0.76      0.60      0.67        89

    accuracy              